In [1]:
import phidl.geometry as pg
import matplotlib.pyplot as plt
%matplotlib qt
from phidl.quickplotter import quickplot2 as qp
import shapely as sh
import numpy as np
from scipy.spatial import cKDTree
import sys
sys.path.append('./')
import utilities as util
import phidl.path as pth


In [2]:
elec_diam=30
ring_width = 20
n_arms = 8
arm_length = 3000
ring_spacing = 300


In [3]:
center = sh.Point(0,0).buffer(elec_diam/2+10)
def ring(radius,width,res=100):
    t=np.linspace(0,2*np.pi,res)
    x=np.cos(t)*radius
    y=np.sin(t)*radius
    return sh.LineString( np.asarray([x,y]).T).buffer(width/2)
def arm(w_in,w_out,mid_l,arm_l):
    in_ring = sh.Polygon([[-w_in/2,0],[w_in/2,0],[w_out/2,mid_l],[-w_out/2,mid_l]])
    out_ring = sh.Polygon([[w_out/2,mid_l],[w_out/2,arm_l],[-w_out/2,arm_l],[-w_out/2,mid_l]])
    
    return in_ring.union(out_ring)
polygons=[]
polygons+=[center]
for n_ring in range(4):
    polygons.append(ring(radius=(n_ring+1)*ring_spacing,width=ring_width))
for idx in range(n_arms):
    polygons.append(sh.affinity.rotate(arm(30,100,4*ring_spacing,arm_l=arm_length),origin=(0,0), angle=360//n_arms*idx))
polygons.append(ring(radius=arm_length,width=50))

In [4]:
sh_dev=sh.unary_union(polygons).buffer(5).buffer(-5)

In [5]:
#place electrodes


In [6]:
n_elecs = [1,8,12,16,24]
angles = [0,22.5,0,11.25,7.5] #degrees


all_elecs_xy=[]

for i in [0,1,2,4]:
    for n in range(n_elecs[i]):
        angle_around_ring = (angles[i]+360//n_elecs[i]*n)*np.pi/180
        radius = i*ring_spacing
        x,y= radius*np.cos(angle_around_ring),radius*np.sin(angle_around_ring)
        electrode = pg.circle(radius=elec_diam/2,layer=2).movex(x).movey(y)
        electrode = pg.circle(radius=elec_diam/2+5,layer=1).movex(x).movey(y)
        sh_dev=sh_dev.union(sh.Point(x,y).buffer(elec_diam/2+5))
        all_elecs_xy.append([x,y])
i = 3
for n in range(n_elecs[i]//2):
    for pm in [-1,1]:
        angle_around_ring = (pm*angles[i]+360//(n_elecs[i]//2)*n)*np.pi/180
        radius = i*ring_spacing
        x,y= radius*np.cos(angle_around_ring),radius*np.sin(angle_around_ring)
        electrode = pg.circle(radius=elec_diam/2,layer=2).movex(x).movey(y)
        electrode = pg.circle(radius=elec_diam/2+5,layer=0).movex(x).movey(y)
        sh_dev=sh_dev.union(sh.Point(x,y).buffer(elec_diam/2+5))
        all_elecs_xy.append([x,y])

In [7]:
ph_dev = util.shapely_to_phidl(sh_dev)
qp(ph_dev)

In [8]:
router = util.RoutingLayout(sh_dev,border_tolerance=5)


In [9]:
router.lay_down_lines(n_lines=7,gap_um=5,adaptive=False)

In [10]:
list_of_bds, list_of_branches = [],[]
for x,y in all_elecs_xy:
    boundary=sh.Point(x,y).buffer(5).exterior
    norm=x*x+y*y+1e-3 #in case centered on 0,0
    nx,ny=x/np.sqrt(norm),y/np.sqrt(norm) #normalized direction
    branch_points=sh.MultiPoint([[x+15*nx,y+15*ny],[x-15*nx,y-15*ny],[x+15*ny,y-15*nx],[x-15*ny,y+15*nx]])
    list_of_bds.append(boundary)
    list_of_branches.append(branch_points)
router.add_electrodes(list_of_bds,list_of_branches)

In [11]:
angle=np.linspace(0,2*np.pi,len(all_elecs_xy)+1)[:-1]
rad=3020
list_of_bds, list_of_branches = [],[]
for a in angle:
    x,y=rad*np.sin(a),rad*np.cos(a)
    list_of_bds.append(sh.Point(x,y).buffer(1).exterior)
    list_of_branches.append(sh.MultiPoint([[(rad-45)*np.sin(a),(rad-45)*np.cos(a)]]))
router.add_electrodes(list_of_bds,list_of_branches,type='pad')

In [12]:
router.route(resample_for_smooth=10)

In [13]:
router.all_elec_pad_routes_smooth

In [14]:
elec_routes=util.multilinestrings_to_phidl(sh.unary_union(router.all_elec_pad_routes_smooth),width=1)

In [15]:
full_device=pg.Device()
full_device<<ph_dev

elecs=pg.Device()
for x,y in all_elecs_xy:
    elecs<<pg.circle(radius=12,layer=1).movex(x).movey(y)
full_device<<elec_routes.remap_layers({3:1})
full_device<<elecs
full_device<<pg.boolean(ph_dev,elecs,operation='A-B',layer=2)

In [16]:
qp(full_device)

In [17]:
full_device.write_gds('Tutorial_dev.gds')

In [18]:
tmp=pg.Device()
tmp<<ph_dev
tmp<<routez
qp(tmp)

In [19]:
# def smoothen_line(sh_line,factor):
#     from scipy.interpolate import splprep,splev
#     xy=np.asarray(sh_line.xy).T
plt.plot(*line.buffer(rad+1e-3).buffer(-rad).interiors[0].xy)
plt.plot(*newline.xy)

In [ ]:
# def buffer_varying(xy,distances,tolerance=.1):
#     '''If creating a loop via xy + normal*distance creates hairpins, the buffering operations may be unstable
#     These hairpins can be detected via checking if the linestring is simple (no self intersection). If it is not
#     we revert to this slower way of building the outline, buffering each point with a specific radius and taking their union
#     The underlying curve (x,y) should be sample more densely than the smallest radius (i.e, the line-line gap)
#     In order to speed up the algorithm, tolerance treats radii within this tolerance as identical, so that buffer can run as
#     a single radius on contiguous poinst (linestrings) of radii that are similar, reducing the number of buffering operations'''
    
#     contig=distances//0.1
#     out = np.split(contig, np.where(contig[1:] != contig[:-1])[0] + 1)
#     slicer=np.cumsum([0]+[len(el) for el in out])
#     bleb_lines=sh.unary_union([sh.LineString(xy[slicer[i]:slicer[i+1]]).buffer(np.mean(distances[slicer[i]:slicer[i+1]])) for i in range(len(slicer)-1) if (slicer[i+1]-slicer[i])>1])
#     bleb_points=sh.unary_union([sh.Point(xy[slicer[i]:slicer[i+1]]).buffer(np.mean(distances[slicer[i]:slicer[i+1]])) for i in range(len(slicer)-1) if (slicer[i+1]-slicer[i])==1])
#     print(slicer)
#     return bleb_lines.union(bleb_points)

In [20]:
sh_line = pol.interiors[0]
from scipy.interpolate import splprep,splev
xy=np.asarray(sh_line.xy).T
s=np.cumsum(np.hstack(np.sum(np.gradient(xy,axis=0)**2,axis=1)))
s=s/np.max(s)
smooth=100
new_s = np.linspace(0,1,10000)
tck,u =splprep(np.vstack((s,xy[:,0])), s=smooth)
_,smooth_x = splev(new_s,tck)
tck,u =splprep(np.vstack((s,xy[:,1])), s=smooth)
_,smooth_y = splev(new_s,tck)

plt.plot(xy[:,0],xy[:,1])
plt.plot(smooth_x,smooth_y)

In [ ]:
# import pygmsh
# import gmsh
# def remap(linestring,res=3):
#     length=linestring.length
#     linestring=sh.LineString(linestring.interpolate(np.linspace(0,1,int(length//res)),normalized=True))
#     return linestring
# shapely_dev=router.shaved_device
# with pygmsh.occ.Geometry() as geom:
#     perimeter=geom.add_polygon( np.asarray(remap(shapely_dev.exterior,3).coords)[:-1,:])
#     #holes= [geom.add_polygon(reshape(d,45)) for d in neg_holes.get_polygons()]
#     holes = [geom.add_polygon(np.asarray(remap(interior,3).coords)[:-1,:])
#              for interior in shapely_dev.interiors]
#     geom.boolean_difference(perimeter, holes)
#    # gmsh.option.setNumber('Mesh.Algorithm',1)
#     #gmsh.option.setNumber("LcMin", 5)

# #     gmsh.option.setNumber('Mesh.Smoothing',1)
# #     gmsh.option.setNumber('Mesh.SubdivisionAlgorithm', 1)
# #     gmsh.option.setNumber('Mesh.RecombineAll', 1)
#     mesh =geom.generate_mesh()